# Основи роботи з LangChain

LangChain — це фреймворк, створений для полегшення побудови застосунків на основі великих мовних моделей (LLMs). Основною ідеєю LangChain є інтеграція LLM у більш складні та персоналізовані робочі процеси, що дозволяє використовувати мовні моделі не просто для генерації тексту, але і для виконання складних бізнес-завдань або автоматизації процесів.

LangChain дозволяє поєднувати LLM з зовнішніми інструментами, базами даних або API. Завдяки цьому можна створювати складніші сценарії використання, наприклад:

1. **Інтеграція з базами даних**: LangChain може працювати з великими обсягами даних або підключатися до зовнішніх джерел інформації, надаючи змогу LLM доступати до актуальних або специфічних даних.
  
2. **Запам'ятовування контексту**: Зазвичай LLM не має "пам'яті", але LangChain дозволяє зберігати стан і контекст між запитами, що корисно для чат-ботів або додатків, які вимагають пам’яті про попередні запити або сесії, однак LangChain може допомогти реалізувати механізми збереження контексту розмови або дій. Це дозволяє моделі пам’ятати попередні відповіді або завдання, роблячи роботу з LLM більш цілісною і логічною в довгих сесіях.

3. **Розгалужені ланцюги дій**: За допомогою LangChain можна побудувати ланцюжки дій, де LLM виконує серію кроків або запитів, заснованих на попередніх результатах. Це корисно для складних задач, які вимагають кількох етапів обробки даних або прийняття рішень.

4. **Інтеграція з іншими інструментами**: LangChain дозволяє легко підключати сторонні інструменти, такі як інтерфейси користувача, системи аналітики або автоматизації бізнес-процесів, що розширює можливості моделі.

5. **Індивідуалізація застосунків**: Розробники можуть створювати спеціалізовані програми для виконання певних завдань на основі даних або бізнес-потреб, таких як чат-боти, автоматизовані системи підтримки клієнтів або моделі рекомендацій.

Таким чином, **LangChain** — це інструмент, який робить LLM більш гнучкими та функціональними в реальних умовах, дозволяючи будувати застосунки на основі штучного інтелекту з урахуванням складних сценаріїв і бізнес-потреб.

Встановлення всіх необхідних пакетів...

In [ ]:
!pip -q install langchain langchain_openai huggingface_hub openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 6.9 MB/s eta 0:00:00


In [ ]:
!pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


#### Налаштування моделі OpenAI

Нам потрібно завантажити деякі токени. Я помістила їх у файл наступної структури:
```
{
"OPENAI_API_KEY": "...",
"HUGGINGFACEHUB_API_TOKEN": "...",
"SERPAPI_API_KEY":"..."
}

```
Потім я просто додала цей файл до контексту цього ноутбука.

In [ ]:
import json
import os

with open('creds.json') as file:
  creds = json.load(file)

os.environ["OPENAI_API_KEY"] = creds["OPENAI_API_KEY"]

Для більшості LLM (великі мовні моделі) ми можемо налаштувати параметр температури, який контролює креативність тексту, що генерується API OpenAI. Вища температура призведе до більш креативного тексту, тоді як нижча температура створить більш передбачуваний текст.

Давайте зробимо наші виходи досить передбачуваними, але з невеликою часткою креативності.

Значення за замовчуванням для температури зазвичай становить 0.7.

In [ ]:
overal_temperature = 0.1

Ви можете знайти опис параметрів моделей OpenAI [тут](https://python.langchain.com/docs/concepts/#chat-models).

А загальну інформацію щодо моделей OpenAI з інтеграцією LangChain - [тут](https://python.langchain.com/docs/integrations/llms/openai)

Сорс код класа `ChatOpenAI`: [тут](https://github.com/langchain-ai/langchain/blob/0640cbf2f126f773b7ae78b0f94c1ba0caabb2c1/libs/community/langchain_community/chat_models/openai.py#L180)

## Різні OpenAI моделі

У фреймворку **LangChain** існують дві основні класи для роботи з моделями OpenAI: **ChatOpenAI** та **OpenAI**. Ось їх основні відмінності:

### 1. **OpenAI** (клас для роботи з мовними моделями):
- **OpenAI** використовується для взаємодії з текстовими мовними моделями, такими як GPT-3 або GPT-4, які приймають текст як вхід і генерують текст у відповідь.
- Цей клас добре підходить для виконання стандартних запитів, де ви передаєте текст і отримуєте текстовий результат.
- **OpenAI** не підтримує багаторівневу структуру розмови чи контекстний діалог; це просто запит-відповідь без збереження історії діалогу.


### 2. **ChatOpenAI** (клас для роботи з чат-моделями):
- **ChatOpenAI** використовується для роботи з моделями, оптимізованими для чат-сценаріїв, такими як GPT-4, які можуть вести розмови та обробляти кілька повідомлень з діалогу.
- Цей клас дозволяє працювати з історією чату, надаючи можливість передавати повідомлення як частину розмови, включаючи повідомлення від користувача (Human) і відповіді моделі (AI).
- **ChatOpenAI** також краще підходить для додатків, де важливо зберігати контекст діалогу та попередні повідомлення.

Таким чином, **ChatOpenAI** краще використовувати, коли потрібно будувати чат-ботів або вести розмову, тоді як **OpenAI** підходить для одноразових генерацій тексту на основі вхідного запиту.

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=overal_temperature)

In [ ]:
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x7bb059ad39b0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7bb05818fb60>, temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

### Отримання прогнозів

In [ ]:
llm.invoke('who am I talking to?')

'\n\nYou are talking to a computer program designed to respond to your questions and prompts.'

In [ ]:
request = "What are 5 vacation destinations for someone who likes to eat pasta?"
print(llm.invoke(request))



1. Italy - The birthplace of pasta, Italy offers a wide variety of pasta dishes from different regions such as spaghetti carbonara, lasagna, and gnocchi.

2. New York City, USA - Known for its diverse food scene, New York City has a plethora of Italian restaurants offering delicious pasta dishes such as fettuccine alfredo and penne arrabiata.

3. Paris, France - While known for its French cuisine, Paris also has a strong Italian influence and offers a range of pasta dishes such as tagliatelle bolognese and linguine with clams.

4. Tokyo, Japan - Surprisingly, Tokyo has a thriving Italian food scene and offers unique fusion dishes such as ramen carbonara and udon bolognese.

5. Buenos Aires, Argentina - With a large Italian immigrant population, Buenos Aires has a strong Italian food culture and offers delicious pasta dishes such as ravioli and spaghetti with meatballs.


А ось таким чином можна реалізувати стрімінг - це якщо ви захочете написати свого чатбота:

In [ ]:
for chunk in llm.stream(
    "What are some theories about the relationship between unemployment and inflation?"
):
    print(chunk, end="", flush=True)

Більше про різні інтерфеси як ранити LLM з langchain - [тут](https://python.langchain.com/docs/integrations/llms/openai/)

### Промпти: Керування запитами для LLMs

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt = PromptTemplate(
    input_variables=["food"],
    template="What are 5 vacation destinations for someone who likes to eat {food}?",
)

In [ ]:
print(prompt.format(food="donuts"))

What are 5 vacation destinations for someone who likes to eat donuts?


In [ ]:
print(llm.invoke(prompt.format(food="donuts")))



1. Portland, Oregon: Known for its vibrant food scene, Portland is home to numerous artisanal donut shops such as Voodoo Doughnut and Blue Star Donuts.

2. New York City, New York: The Big Apple is a haven for foodies, and donut lovers will not be disappointed with the variety of options available, from classic New York-style bagels to trendy gourmet donut shops like Doughnut Plant.

3. Austin, Texas: This hip and eclectic city is home to unique and creative donut shops like Gourdough's and Voodoo Doughnut, as well as traditional Texas-style kolaches.

4. Nashville, Tennessee: In addition to its famous hot chicken, Nashville also boasts a thriving donut scene with popular spots like Five Daughters Bakery and Fox's Donut Den.

5. Chicago, Illinois: The Windy City is known for its deep-dish pizza, but it also has a strong donut game with iconic shops like Glazed and Infused and Do-Rite Donuts.


### Ланцюги: Поєднання LLM (великі мовні моделі) та запитів у багатоступеневих робочих процесах

Організувтати ланцюг дуже просто:

In [ ]:
chain = prompt | llm
print(chain.invoke("fresh fish"))



1. Tokyo, Japan: Known for its world-renowned sushi and sashimi, Tokyo is a must-visit destination for seafood lovers. The Tsukiji Fish Market, the largest fish market in the world, offers a wide variety of fresh fish and seafood.

2. San Sebastian, Spain: This coastal city in the Basque Country is famous for its pintxos (small tapas-style dishes) and fresh seafood. The city's Old Town is filled with traditional seafood restaurants and markets.

3. Cape Town, South Africa: With its location on the coast, Cape Town offers a diverse range of fresh seafood, including local specialties like snoek and crayfish. The city's vibrant food scene also includes a variety of seafood-focused restaurants.

4. Sydney, Australia: As a coastal city, Sydney is known for its fresh seafood, particularly its famous Sydney rock oysters. The city's fish markets and waterfront restaurants offer a wide selection of fresh fish and seafood dishes.

5. Vancouver, Canada: Located on the Pacific coast, Vancouver i

### Агенти: Можливість динамічно викликати ланцюги на основі введення користувача

**Агент**

Як зазначено в документації, Агент в LangChain охоплює наступні абстракції:

1. **AgentAction**: Представляє наступну дію, яку необхідно виконати, що складається з інструменту та вхідних даних для інструменту.
   
2. **AgentFinish**: Остаточний результат від агента, що містить фінальний результат агента в `return_values`.
   
3. **Intermediate Steps**: Означає попередні дії агента та їх відповідні результати, організовані як список кортежів `[AgentAction, Any]`.

Ця структура вказує на те, що для запиту може бути виконано кілька дій агента за потреби, при цьому проміжні дії зберігаються в проміжних кроках.

**AgentExecutor**

AgentExecutor відповідає за використання Агента до тих пір, поки не буде отримано остаточний результат. Таким чином, він використовує Агента для отримання наступної дії, виконує повернуту дію поетапно та продовжує цей процес, доки не буде згенеровано остаточну відповідь для даного вводу.

Раптом вам цікаов, як влаштовані технічно агенти під капотом:
![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*CD2Svi5BUZaG-d-KWeU3ug.png)

Детально про те, як під капотом працюють агенти можна прочитати [тут](https://nakamasato.medium.com/langchain-how-an-agent-works-7dce1569933d).

In [ ]:
!pip install -q google-search-results langchain-community langchain_experimental

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


Щоб зробити API пошуку доступним (100 пошуків на місяць), просто зареєструйтесь [тут](https://serpapi.com/users/welcome).
Ми будемо використовувати [ReAct](https://react-lm.github.io/) агента. ReAct від Reasoning-Action. Агенти ReAct від LangChain допомагають організувати весь процес обробки запитів. Використовуючи ці агенти, ми можемо розбити складні запити на керовані кроки і виконувати їх систематично. Агент задається спеціальним промптом, який ми далі з вами розглянемо.

In [ ]:
from langchain import hub
from langchain.agents import load_tools
from langchain.agents import Tool, AgentExecutor, AgentType, create_react_agent, initialize_agent
from langchain.chains import LLMMathChain
from langchain_experimental.utilities import PythonREPL

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=overal_temperature)
problem_chain = LLMMathChain.from_llm(llm=llm)
math_tool = Tool.from_function(name="Calculator",
                func=problem_chain.run,
                description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions. Cannot perfor max/min/prime number search operations.")

In [ ]:
problem_chain.invoke('(112*132)-19/3')

{'question': '(112*132)-19/3', 'answer': 'Answer: 14777.666666666666'}

In [ ]:
llm.invoke('(112*132)-19/3')

AIMessage(content='To solve the expression \\( (112 \\times 132) - \\frac{19}{3} \\), we first calculate \\( 112 \\times 132 \\):\n\n\\[\n112 \\times 132 = 14784\n\\]\n\nNext, we calculate \\( \\frac{19}{3} \\):\n\n\\[\n\\frac{19}{3} \\approx 6.3333\n\\]\n\nNow, we subtract \\( \\frac{19}{3} \\) from \\( 14784 \\):\n\n\\[\n14784 - \\frac{19}{3} = 14784 - 6.3333 \\approx 14777.6667\n\\]\n\nThus, the final result is approximately:\n\n\\[\n14777.67\n\\] \n\nIf you need the exact value in fractional form, it would be:\n\n\\[\n14784 - \\frac{19}{3} = \\frac{44352 - 19}{3} = \\frac{44333}{3}\n\\]\n\nSo, the final answer can be expressed as:\n\n\\[\n\\frac{44333}{3} \\text{ or approximately } 14777.67\n\\]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 236, 'prompt_tokens': 15, 'total_tokens': 251, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_to

In [ ]:
(112*132)-19/3

14777.666666666666

REPL (Read-Eval-Print Loop) — це інтерактивне середовище програмування, яке дозволяє користувачу вводити код, отримувати його результат і продовжувати виконання в режимі реального часу. По суті, це цикл, який "читає" введення, "оцінює" його, "друкує" результат і чекає на новий введений код. REPL часто використовується в мовах програмування для швидкого тестування й налагодження коду, таких як Python, JavaScript тощо.

In [ ]:
python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you necessarily should print it out with `print(...)`. Otherwise you won't see the result! It's very important.",
    func=python_repl.run,
)
python_tool.name = "python_interpreter"

In [ ]:
python_repl.run("import pandas as pd; s = pd.Series([1,2,3])")

''

In [ ]:
os.environ["SERPAPI_API_KEY"] = creds["SERPAPI_API_KEY"]

In [ ]:
tools = load_tools(["serpapi"], llm=llm)
prompt = hub.pull("hwchase17/react")

In [ ]:
tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='5d906dfd010009d73a4a602a0ad8ac770f46dc2077bfd05ed5e4ffe4471ca2ef', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='5d906dfd010009d73a4a602a0ad8ac770f46dc2077bfd05ed5e4ffe4471ca2ef', aiosession=None)>)]

In [ ]:
tools.append(math_tool)
tools.append(python_tool)

In [ ]:
# tools

In [ ]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Дивіться список типів агентів [тут](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)

In [ ]:
agent_executor.invoke({'input': "Who is the current leader of China? What is the largest prime number that is smaller than their age?"})



> Entering new AgentExecutor chain...
To answer the question, I first need to find out who the current leader of China is. After that, I will determine their age and find the largest prime number smaller than that age.

Action: Search  
Action Input: "current leader of China 2023"  
['In October 2022, Xi secured a third term as CCP General Secretary, and was re-elected state president for an unprecedented third term in March 2023.', 'Xi Jinping is a politician and government official who became president of China in 2013 and general secretary of the Chinese Communist Party in 2012. He was ...', "President of China ; Incumbent Xi Jinping. since 14 March 2013. Office of the President of the. People's Republic of China ; Incumbent Xi Jinping. since 14 March ...", "Xi Jinping · The leader of the world's most populated country and second largest economy (GDP), Xi Jinping holds a firm grasp on China's only political party.", "China's leader Xi Jinping has secured a historic third term as p

{'input': 'Who is the current leader of China? What is the largest prime number that is smaller than their age?',
 'output': 'The current leader of China is Xi Jinping, and the largest prime number smaller than his age (70) is 67.'}

Агенти мають особиливий тип:

In [ ]:
type(agent)

langchain_core.runnables.base.RunnableSequence

Він створюється за допомогою RunnableAssign, PromptTemplate, RunnableBinding та ReActSingleInputOutputParser, структурованих у формі [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/concepts/#langchain-expression-language-lcel)

In [ ]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': "Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions. Cannot perfor max/min/prime number search operations.\npython_interpreter(command: str, timeout: Optional[int] = None) -> str - A Python shell. Use this to execute python commands. Input should be a valid py

Перший крок — це RunnableAssign, який відповідає за присвоєння пар "ключ-значення" вхідним даним формату Dict[str, Any]. У цьому випадку ключем є agent_scratchpad, а значенням є RunnableLambda, що перетворює intermediate_steps на рядок.

In [ ]:
agent.steps[1].partial_variables

{'tools': "Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions. Cannot perfor max/min/prime number search operations.\npython_interpreter(command: str, timeout: Optional[int] = None) -> str - A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you necessarily should print it out with `print(...)`. Otherwise you won't see the result! It's very important.",
 'tool_names': 'Search, Calculator, python_i

Ми ще і промпти на кожному етапі можемо переглянути.

In [ ]:
print(agent.steps[1].template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Пам'ять
Часто нам хочеться, аби LLM памʼятала історію діалогу. Найпростіша форма пам'яті - це просто передача повідомлень історії чату по ланцюжку. Ось приклад:


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)
print(ai_msg)

content='I said "J\'adore la programmation," which translates to "I love programming" in French.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 56, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CG8pwUhhAF5BPa9hzWiJuMypXJIFz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--9b1bdbcd-8928-466c-a3ba-656e1445635b-0' usage_metadata={'input_tokens': 56, 'output_tokens': 19, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
ai_msg.content

'I said "J\'adore la programmation," which translates to "I love programming" in French.'

Ми бачимо, що, передаючи попередню розмову по ланцюжку, він може використовувати її як контекст для відповідей на запитання. Це основна концепція, що лежить в основі пам'яті чат-ботів - решта посібника продемонструє зручні прийоми для передачі або переформатування повідомлень.


## Автоматичне керування історією

У попередніх прикладах повідомлення передаються в ланцюжок (та модель) явно. Це абсолютно прийнятний підхід, але він вимагає зовнішнього керування новими повідомленнями. LangChain також надає спосіб створення застосунків, що мають пам'ять, використовуючи [persistence](https://langchain-ai.github.io/langgraph/concepts/persistence/) LangGraph. Ви можете увімкнути persistence у застосунках LangGraph, надавши контрольну точку (checkpointer) під час компіляції графа.



In [ ]:
! pip install -q langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 932.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 9.5 MB/s eta 0:00:00


In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

# Визначаємо функцію, яка викликає модель
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": response}


# Визначаємо вузол та ребро
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Додаємо простий контроль стану в пам'яті
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
app.invoke(
    {"messages": [HumanMessage(content="Translate to French: I love programming.")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='Translate to French: I love programming.', additional_kwargs={}, response_metadata={}, id='21ec9a11-0de2-4c97-9a81-9b8f06b8091c'),
  AIMessage(content="J'adore programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 35, 'total_tokens': 39, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a7bd92b-63bc-4059-a542-9dc2668f7489-0', usage_metadata={'input_tokens': 35, 'output_tokens': 4, 'total_tokens': 39, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}

In [ ]:
app.invoke(
    {"messages": [HumanMessage(content="What did I just ask you?")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='Translate to French: I love programming.', additional_kwargs={}, response_metadata={}, id='21ec9a11-0de2-4c97-9a81-9b8f06b8091c'),
  AIMessage(content="J'adore programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 35, 'total_tokens': 39, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a7bd92b-63bc-4059-a542-9dc2668f7489-0', usage_metadata={'input_tokens': 35, 'output_tokens': 4, 'total_tokens': 39, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
  HumanMessage(content='What did I just ask you?', additional_kwargs={}, response_metadata={}, id='01b3f193-5f40-4103-a639-093e4149ff1c'),
  AIMessage(content='You asked me 

# Зміна історії чату
Зміна збережених повідомлень чату може допомогти вашому чат-боту впоратися з різними ситуаціями.

## Обрізання повідомлень
LLM і чат-моделі мають обмежені контекстні вікна, і навіть якщо ви не перевищуєте ліміти, ви можете обмежити кількість відволікаючих чинників, з якими доводиться мати справу моделі. Одне з рішень - обрізати повідомлення історії перед тим, як передавати їх моделі. Давайте розглянемо приклад історії з додатком, який ми оголосили вище:


In [ ]:
demo_ephemeral_chat_history = [
    HumanMessage(content="Hey there! I'm Nemo."),
    AIMessage(content="Hello!"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="Fine thanks!"),
]

app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage(content="What's my name?")]
    },
    config={"configurable": {"thread_id": "2"}},
)

{'messages': [HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}, id='e5054b65-d7ee-477a-9519-027e923206fb'),
  AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='0d22c145-cc6d-4cea-a492-3318001cf2c8'),
  HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='6e6ca81c-6c4e-4d8f-a5e2-9377459909bd'),
  AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='e71319c0-aefb-4858-bb33-91e1452289d1'),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='19ee0ae3-fc42-486c-863a-669701ac6c89'),
  AIMessage(content='Your name is Nemo!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 63, 'total_tokens': 68, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-m

Ми бачимо, що програма запам'ятовує попередньо завантажене ім'я.

Але уявімо, що у нас дуже маленьке контекстне вікно, і ми хочемо зменшити кількість повідомлень, що передаються моделі, до 2 останніх. Ми можемо використати вбудовану утиліту trim_messages для відсікання повідомлень на основі їх кількості токенів до того, як вони досягнуть нашого запиту. У цьому випадку ми будемо вважати кожне повідомлення за 1 «токен» і залишимо лише два останніх повідомлення:



In [ ]:
from langchain_core.messages import trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Визначаємо функцію обрізання повідомлень
# рахуємо кожне повідомлення як 1 "токен" (token_counter=len) і залишаємо лише останні два повідомлення
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

workflow = StateGraph(state_schema=MessagesState)

# Визначаємо функцію, яка викликає модель
def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + trimmed_messages
    response = llm.invoke(messages)
    return {"messages": response}


# Визначаємо вузол та ребро
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Додаємо простий контроль стану в пам'яті
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [ ]:
app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage(content="What is my name?")]
    },
    config={"configurable": {"thread_id": "3"}},
)

{'messages': [HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}, id='e5054b65-d7ee-477a-9519-027e923206fb'),
  AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='0d22c145-cc6d-4cea-a492-3318001cf2c8'),
  HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='6e6ca81c-6c4e-4d8f-a5e2-9377459909bd'),
  AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='e71319c0-aefb-4858-bb33-91e1452289d1'),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='a45d3ebd-247d-4191-ae02-7cb01281aa71'),
  AIMessage(content="I'm sorry, but I don't have access to personal information about users unless you provide it. If you'd like, you can tell me your name!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 39, 'total_tokens': 68, 'completion_tokens_details': {'audio_tokens': None

## Саммарі історії повідомлень

Ми можемо використовувати цей промпт і в інших випадках. Наприклад, ми можемо використовувати додатковий виклик LLM для створення резюме розмови перед викликом нашого додатку. Давайте відтворимо нашу історію чату:


In [ ]:
demo_ephemeral_chat_history = [
    HumanMessage(content="Hey there! I'm Nemo."),
    AIMessage(content="Hello!"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="Fine thanks!"),
]

In [ ]:
from langchain_core.messages import HumanMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)


# Функція, яка викликає модель
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability. "
        "The provided chat history includes a summary of the earlier conversation."
    )
    system_message = SystemMessage(content=system_prompt)
    message_history = state["messages"][:-1]  # вилучаємо найостанніше введення
    # Підсумовуємо повідомлення, якщо історія чату досягає певного розміру
    if len(message_history) >= 4:
        last_human_message = state["messages"][-1]
        # Викликаємо модель для створення підсумкових повідомлень розмови
        summary_prompt = (
            "Distill the above chat messages into a single summary message. "
            "Include as many specific details as you can."
        )
        summary_message = llm.invoke(
            message_history + [HumanMessage(content=summary_prompt)]
        )

        # Видаляємо повідомлення, які більше не хочемо відображати
        delete_messages = [RemoveMessage(id=m.id) for m in state["messages"]]
        # Повторно додаємо повідомлення від користувача
        human_message = HumanMessage(content=last_human_message.content)
        # Викликаємо модель для обробки підсумкових повідомлень і відповіді
        response = llm.invoke([system_message, summary_message, human_message])
        message_updates = [summary_message, human_message, response] + delete_messages
    else:
        message_updates = llm.invoke([system_message] + state["messages"])

    return {"messages": message_updates}


# Визначаємо вузол та ребро
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Додаємо простий контроль стану в пам'яті
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [ ]:
app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage("What did I say my name was?")]
    },
    config={"configurable": {"thread_id": "4"}},
)

{'messages': [AIMessage(content='Nemo greeted me with "Hey there!" and asked how I was doing, to which I responded that I was fine.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 60, 'total_tokens': 85, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-df5ffc76-77c8-4a2a-923b-31edac843e88-0', usage_metadata={'input_tokens': 60, 'output_tokens': 25, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
  HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={}, id='e160fac5-f4a2-4dca-92e2-54aa32fe8c52'),
  AIMessage(content='You mentioned that your name is Nemo.', additional_kwargs={'refusal': None}, respons

Зауважте, що повторний запуск програми продовжить накопичення історії доти, доки вона не досягне вказаної кількості повідомлень (у нашому випадку - чотирьох). У цей момент ми створимо ще один звіт, згенерований з початкового звіту плюс нові повідомлення, і так далі.


# Порівняння та оцінка LLM (Large Language Models)

Зручно мати механізм порівняти кілька LLM. Для порівняння використаємо моделі з HuggingFace. Всі моделі можна знайти [тут](https://huggingface.co/models)

**Hugging Face** — це платформа та компанія, яка спеціалізується на штучному інтелекті, зокрема на роботі з мовними моделями та NLP (обробкою природної мови). Вони створили популярну бібліотеку **Transformers**, яка надає інструменти для роботи з передовими моделями штучного інтелекту, такими як GPT, BERT, T5, та інші. Hugging Face також має платформу для спільного використання моделей, де розробники можуть завантажувати, тестувати та використовувати попередньо навчені моделі для різних задач, таких як текстова класифікація, генерація тексту та переклад.

In [ ]:
! pip install -q langchain-huggingface

In [ ]:
! pip freeze | grep huggingface

huggingface-hub==0.34.4
langchain-huggingface==0.3.1


In [ ]:
os.environ["HF_TOKEN"] = creds['HUGGINGFACEHUB_API_TOKEN']

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
# test
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    huggingfacehub_api_token="my-api-key",
    task='conversational'
)
print(llm.invoke("What is Deep Learning?", ))

ValueError: Model mistralai/Mistral-7B-Instruct-v0.2 is not supported for task text-generation and provider featherless-ai. Supported task: conversational.

Після запису лекції сталась зміна в бібліотеці HuggingFace і попередній варіант роботи з моделями не працює з  причин, описаних в коментарі тут: https://github.com/langchain-ai/langchain/issues/31434#issuecomment-2936308959

Тому пропонується натомість використовувтаи моделі так:

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import os

llm = HuggingFaceEndpoint(
  repo_id="mistralai/Mistral-7B-Instruct-v0.3",
  huggingfacehub_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

chat_model = ChatHuggingFace(llm=llm)

prompt = ChatPromptTemplate.from_messages([
  SystemMessagePromptTemplate.from_template("You're a helpful assistant."),
  HumanMessagePromptTemplate.from_template("{user_question}"),
])

chain = prompt | chat_model

response = chain.invoke({"user_question": "What happens when an unstoppable force meets an immovable object?"})

print(response.content)

 According to a common philosophical thought experiment, when an unstoppable force meets an immovable object, neither the force nor the object would be able to move. This leads to a paradox, as both cannot be stopped, yet they cannot move each other. However, it's important to note that this is a thought experiment and doesn't have a definitive answer in the physical world.


З цікавого ще - gemma не має прописаних провайдерів, а mistral має.

In [ ]:
from huggingface_hub import HfApi, ModelInfo

model_id = "mistralai/Mistral-7B-Instruct-v0.3" # Specify the model ID

model_info: ModelInfo = HfApi().model_info(
    model_id, expand=["inferenceProviderMapping"]
)

model_info.inference_provider_mapping

[InferenceProviderMapping(provider='together', hf_model_id='mistralai/Mistral-7B-Instruct-v0.3', provider_id='mistralai/Mistral-7B-Instruct-v0.3', status='live', task='conversational', adapter=None, adapter_weights_path=None, type=None),
 InferenceProviderMapping(provider='novita', hf_model_id='mistralai/Mistral-7B-Instruct-v0.3', provider_id='mistralai/mistral-7b-instruct', status='live', task='conversational', adapter=None, adapter_weights_path=None, type=None)]

In [ ]:
model_id = "google/gemma-7b" # Specify the model ID

model_info: ModelInfo = HfApi().model_info(
    model_id, expand=["inferenceProviderMapping"]
)

model_info.inference_provider_mapping

[]

Тому будемо використовувати 2 інші модельки - llama та qwen.

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct" # Specify the model ID
model_id = "Qwen/Qwen2.5-7B-Instruct"

model_info: ModelInfo = HfApi().model_info(
    model_id, expand=["inferenceProviderMapping"]
)

model_info.inference_provider_mapping

[InferenceProviderMapping(provider='together', hf_model_id='Qwen/Qwen2.5-7B-Instruct', provider_id='Qwen/Qwen2.5-7B-Instruct-Turbo', status='live', task='conversational', adapter=None, adapter_weights_path=None, type=None),
 InferenceProviderMapping(provider='featherless-ai', hf_model_id='Qwen/Qwen2.5-7B-Instruct', provider_id='Qwen/Qwen2.5-7B-Instruct', status='live', task='conversational', adapter=None, adapter_weights_path=None, type=None)]

In [ ]:
# цей код наразі не працює

# overal_temperature = 0.1

# mistral = HuggingFaceEndpoint(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.2",
#     temperature=overal_temperature,
#     max_new_tokens=200
# )

# gemma = HuggingFaceEndpoint(
#     repo_id="google/gemma-7b",
#     temperature=overal_temperature,
#     max_new_tokens=500
# )

In [ ]:
overal_temperature = 0.1

mistral_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=overal_temperature,
    max_new_tokens=200
)
mistral = ChatHuggingFace(llm=mistral_llm)

llama_llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    temperature=overal_temperature,
    max_new_tokens=500
)
llama = ChatHuggingFace(llm=llama_llm)

qwen_llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    temperature=overal_temperature,
    max_new_tokens=500
)
qwen = ChatHuggingFace(llm=qwen_llm)

## Налаштування лабораторії для порівняння

In [ ]:
from langchain.model_laboratory import ModelLaboratory

In [ ]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
models_list = [
    mistral,
    llama,
    qwen
]

In [ ]:
lab = ModelLaboratory.from_llms(models_list, prompt=prompt)

Давайте запустимо це на деяких і порівняємо!

In [ ]:
lab.compare("What is the opposite of up?")

Input:
What is the opposite of up?

llm=HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', max_new_tokens=200, temperature=0.1, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>) model_id='mistralai/Mistral-7B-Instruct-v0.2' model_kwargs={}
 The opposite of "up" is typically considered to be "down." When an object or a person moves from a higher position to a lower position, we often describe that movement as going "down." Conversely, moving from a lower position to a higher position is described as going "up." So, if you're looking for a word that is the polar opposite of "up," then "down" is the answer you're looking for.

llm=HuggingFaceEndpoint(repo_id='meta-llama/Llama-3.1-8B-Instruct', max_new_tokens=500, temperature=0.1, stop_sequences=[], server_kw

In [ ]:
lab.compare("Answer the following question by reasoning step by step. The cafeteria had 23 apples. \
If they used 20 for lunch, and bought 6 more, how many apple do they have?")

Input:
Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?

llm=HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', max_new_tokens=200, temperature=0.1, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>) model_id='mistralai/Mistral-7B-Instruct-v0.2' model_kwargs={}
 The cafeteria started with 23 apples. They used 20 apples for lunch, so that leaves us with:

23 apples (initial amount) - 20 apples (used for lunch) = 3 apples remaining

Then they bought an additional 6 apples. So the new total is:

3 apples (remaining) + 6 apples (newly purchased) = 9 apples

Therefore, the cafeteria has 9 apples.

llm=HuggingFaceEndpoint(repo_id='meta-llama/Llama-3.1-

In [ ]:
lab.compare('''
  Can Elon Musk have a conversation with George Washington? Give the rationale before answering.
''')

Input:

  Can Elon Musk have a conversation with George Washington? Give the rationale before answering.


llm=HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', max_new_tokens=200, temperature=0.1, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>) model_id='mistralai/Mistral-7B-Instruct-v0.2' model_kwargs={}
 I'd be happy to help answer your question, but it's important to note that Elon Musk and George Washington were historical figures who lived in different time periods. Elon Musk is a business magnate, industrial designer, and engineer who was born in 1971, while George Washington was a military leader, statesman, and the first President of the United States, who was born in 1732.

It's not possible for Elon Musk to have a conversation with George Wa

Давайте змінимо запит.

In [ ]:
template = """You are a professional social media manager who can write great posts in linkedin to increase appeal of persons profile: {request}

Story:"""
prompt = PromptTemplate(template=template, input_variables=["request"])

lab = ModelLaboratory.from_llms(models_list, prompt=prompt)

In [ ]:
lab.compare('''I have passed a course in large language models (1 month duration). Write a post about that.''')

Input:
I have passed a course in large language models (1 month duration). Write a post about that.

llm=HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', max_new_tokens=200, temperature=0.1, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>) model_id='mistralai/Mistral-7B-Instruct-v0.2' model_kwargs={}
 🚀 Exciting News: I'm thrilled to announce that I've recently completed a 1-month intensive course in Large Language Models! 🔬📚

If you're not familiar, Large Language Models are sophisticated AI systems designed to understand and generate human-like text. They're the technology behind many advanced natural language processing applications, including chatbots, translation tools, and writing assistants.

Throughout the course, I had the privilege of learnin

In [ ]:
template = """Answer the question to the best of your abilities but if you are not sure then answer you don't know: {question}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["question"])

lab = ModelLaboratory.from_llms(models_list, prompt=prompt)

In [ ]:
lab.compare('''I am riding a bicycle. The pedals are moving fast. I look into the mirror and I am not moving. Why is this?''')


Input:
I am riding a bicycle. The pedals are moving fast. I look into the mirror and I am not moving. Why is this?

llm=HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', max_new_tokens=200, temperature=0.1, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>) model_id='mistralai/Mistral-7B-Instruct-v0.2' model_kwargs={}
 This situation could be explaining an optical illusion, often referred to as the "Mirror Riddle." The reason why you may not be moving in the mirror while you're pedaling your bicycle is because the reflection in the mirror shows a stationary image of you pedaling, but in reality, you are continually moving forward with the bicycle. This visual discrepancy can be quite disorienting but does not mean that something is physically preventing y

### Визначення іменованих сутностей

In [ ]:
template = """{question}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["question"])

lab = ModelLaboratory.from_llms(models_list, prompt=prompt)

In [ ]:
lab.compare('''Extract names and cities from the text.\n\n
Output in the format: {"names": list of names in text, "cities": list of cities in text}, surname]\n\n

Mark Dickey, 40, began suffering from severe gastric pain last week after descending
more than 3,600 feet into Morca Cave in Southern Turkey, according to a statement
from the European Cave Rescue Association, but he wasn’t able to be rescued until yesterday,
when doctors deemed him “transportable.”
''')

Input:
Extract names and cities from the text.


Output in the format: {"names": list of names in text, "cities": list of cities in text}, surname]



Mark Dickey, 40, began suffering from severe gastric pain last week after descending
more than 3,600 feet into Morca Cave in Southern Turkey, according to a statement
from the European Cave Rescue Association, but he wasn’t able to be rescued until yesterday,
when doctors deemed him “transportable.”


llm=HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', max_new_tokens=200, temperature=0.1, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>) model_id='mistralai/Mistral-7B-Instruct-v0.2' model_kwargs={}
 {
"names": ["Mark Dickey"],
"cities": ["Southern Turkey"]
}

llm=HuggingFaceEndpoint(repo_id='meta-llama/L

Відповіді на запитання на основі тексту

In [ ]:
lab.compare('''Is Mark Dickey alive?\n\n
Output in the format: Yes or No, facts that prove that.\n\n

Mark Dickey, 40, began suffering from severe gastric pain last week after descending
more than 3,600 feet into Morca Cave in Southern Turkey, according to a statement
from the European Cave Rescue Association, but he wasn’t able to be rescued until yesterday,
when doctors deemed him “transportable.”
''')

Input:
Is Mark Dickey alive?


Output in the format: Yes or No, facts that prove that.



Mark Dickey, 40, began suffering from severe gastric pain last week after descending
more than 3,600 feet into Morca Cave in Southern Turkey, according to a statement
from the European Cave Rescue Association, but he wasn’t able to be rescued until yesterday,
when doctors deemed him “transportable.”


llm=HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', max_new_tokens=200, temperature=0.1, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>) model_id='mistralai/Mistral-7B-Instruct-v0.2' model_kwargs={}
 Yes, Mark Dickey is alive based on the information provided in the statement from the European Cave Rescue Association. The statement indicates that Mark was suffering

Більше прикладів - в репозиторії LangChain.